In [ ]:
# Dependencies to Visualize the model
import numpy as np
import pandas as pd
np.random.seed(0)

# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
     |█████████████████████████████   | 184.9MB 1.3MB/s eta 0:00:16

In [ ]:
pip install yfinance

# Yahoo Finance

In [ ]:
import yfinance as yf

# tickers = yf.Tickers('msft aapl goog')
# # ^ returns a named tuple of Ticker objects

# # access each ticker using (example)
# tickers.tickers.MSFT.info
# tickers.tickers.AAPL.history(period="1mo")
# tickers.tickers.GOOG.actions

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://ml-project3.s3.ca-central-1.amazonaws.com/final_finance_data.csv"
spark.sparkContext.addFile(url)
stock_data_df = spark.read.csv(SparkFiles.get("final_finance_data.csv"), sep=",", header=True)

# Show DataFrame
stock_data_df.show()

AnalysisException: ignored

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
stock_data_df = df.withColumn('length', length(df['text']))

In [ ]:
stock_data_df['Price Change'] = stock_data_df['Close'] - stock_data_df['Open']
stock_data_df['% Price Change'] = stock_data_df['Price Change'] / stock_data_df['Open']

stock_data_df

,Open,High,Low,Close,Adj Close,Volume,Price Change,% Price Change
Date,,,,,,,,
2020-01-02,158.779999,160.729996,158.330002,160.619995,159.352386,22622100,1.839996,0.011588
2020-01-03,158.320007,159.949997,158.059998,158.619995,157.368179,21116200,0.299988,0.001895
2020-01-06,157.080002,159.100006,156.509995,159.029999,157.774948,20813700,1.949997,0.012414
2020-01-07,159.320007,159.669998,157.320007,157.580002,156.336395,21634100,-1.740005,-0.010921
2020-01-08,158.929993,160.800003,157.949997,160.089996,158.826569,27746500,1.160004,0.007299
...,...,...,...,...,...,...,...,...
2020-11-12,217.210007,219.110001,214.460007,215.440002,215.440002,21593900,-1.770004,-0.008149
2020-11-13,216.360001,217.419998,214.160004,216.509995,216.509995,18621100,0.149994,0.000693
2020-11-16,214.869995,217.740005,214.520004,217.229996,217.229996,24953300,2.360001,0.010983


In [ ]:
def hold(column, tolerance=0.01):
  if abs(column) > tolerance and column > 0:
    return 'buy'
  elif abs(column) > tolerance and column < 0:
    return 'sell'
  else: 
    return 'hold'

stock_data_df['action'] = stock_data_df['% Price Change'].apply(hold)

In [ ]:
stock_data_df['action']

Date
2020-01-02     buy
2020-01-03    hold
2020-01-06     buy
2020-01-07    sell
2020-01-08    hold
              ... 
2020-11-12    hold
2020-11-13    hold
2020-11-16     buy
2020-11-17    hold
2020-11-18    sell
Name: action, Length: 224, dtype: object

# NaiveBayes Machine Learning

In [ ]:
# Read in data from file
from pyspark import SparkFiles

## Import upload data into AWS server##
# file name: ml_finance_data

url ="Resources/'aws_url"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("stock_data.csv"), sep=",", header=True)

# Show DataFrame
df.show()

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['text']))
data_df.show()

Feature Transformations

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='action',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

In [ ]:
# Show label and resulting features
cleaned.select(['label', 'features']).show(truncate = False)

NaiveBayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(10)

In [ ]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

In [ ]:
# Save the model
predictor.save('Trade_Predictor_Model')